# 5. Deep Learning for Computer Vision

## Introduction to Convnets

In [4]:
from tensorflow.keras.datasets import mnist

from tensorflow import keras

Using TensorFlow backend.


In [6]:
# Ingestion
###########
(train_data_full, train_labels_full), (test_data, test_labels) = mnist.load_data()
# Preprocessing
###############
x_train = train_data_full.reshape((60000, 28, 28, 1))
x_train = x_train.astype('float32') / 255.0

x_test = test_data.reshape((10000, 28, 28, 1))
x_test = x_test.astype('float32') / 255.0

y_train, y_test = keras.utils.to_categorical(train_labels_full), keras.utils.to_categorical(test_labels)

The following is what a basic convolutional neural network, or <b>convnet</b> looks like. It now consists of `Conv2D` and `MaxPooling2D` layers.

In [2]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model.add(keras.layers.MaxPooling2D(2,2))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(2,2))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

A convnet takes as input tensors of shape `(image_height, image_width, image_channels)`. In this case we configure the convnet to process inputs of size `(28,28,1)` which is the format of the MNIST images. We do this by passing the argument `(28, 28, 1)` to the first layer.

You can see the output of every `Conv2D` and `MaxPooling` layer is a 3D tensor of shape `(height, width, channels)`. The width and height dimensions tend to shrink as you go deeper into the network. The number of channels is controlled by the last argument passed into the `Conv2D` layers.

Next, we feed the last output tensor of shape `(3, 3, 64)` into a densely connected classifier network like what we are already familier with - a stack of `Dense` layers. These classifiers process vectors which are 1D, while the current output is 3D. So we flatten it, followed by addeing the dense layers.

Finally, we will do 10-way classification, using a final layer with 10 outputs and softmax activation.

In [3]:
# Test: See the current architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

Here is the model visualised

<img src="img5a.png" width="125"/>

Notice that the `(3, 3, 64)` output from the `Conv2D` is flattened to a vector of shape `(576,)` before going through the 2 `Dense` layers. Let's now train this on the MNIST digits, i.e. tackling the **handwritten-digit** problem.

In [7]:
# Train
#######
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 35s 37ms/step - loss: 0.1684 - accuracy: 0.9475
Epoch 2/5
938/938 [==============================] - 44s 46ms/step - loss: 0.0471 - accuracy: 0.9853
Epoch 3/5
938/938 [==============================] - 39s 41ms/step - loss: 0.0321 - accuracy: 0.9900
Epoch 4/5
938/938 [==============================] - 37s 39ms/step - loss: 0.0244 - accuracy: 0.9924
Epoch 5/5
938/938 [==============================] - 37s 39ms/step - loss: 0.0198 - accuracy: 0.9939


In [8]:
# Save model
# model.save('model0.h5')

In [10]:
# Evaluation
############
test_loss, test_acc = model.evaluate(x_test, y_test)
print(test_loss, test_acc)

313/313 [==============================] - 4s 11ms/step - loss: 0.0285 - accuracy: 0.9918
0.02849671244621277 0.9918000102043152


This model has an accuracy of 99.1%, beating the dense model's accuracy of 97.8%. It's very good indeed! But why does convnets outperform the MLP? We answer this by understanding the `Conv2D` and `MaxPooling2D` layers.

### The Convolution Operation

The fundamental difference between a Dense layer and a Convolution layer is that Dense layers learn global patterns in their input feature space whereas Convolution layers learn local patterns. In the **handwritten-digit** example, a Dense layer learns from all pixels while a Convolution layer learns through small windows. This results in two properties:

1 - The patterns they learn are <b>translation invariant</b>. Once a feature is learnt on one position, a convnet can recognise it anywhere. In contrast, a densely connected network would have to learn the pattern anew if it appeared in a new location. Hence, a convnet is data efficient when processing images: they need fewer training examples to learn representations that have generalization power.

<img src="img5b.png" width="350"/>

2 - They can learn <b>spatial hierarchies</b> of patterns. A first convolution layer will learn small local patterns such as edges, a second convolution layer will learn larger patterns made of features of the first layers, and so on. This allows convnets to efficiently learn increasingly complex and abstract visual concepts.

<img src="img5c.png" width="350"/>

Convolutions operate over 3D tensors, called feature maps, with 2 spatial axes (height & width) and a depth axis (channels). For an RGB image, the dimension of the depth axis is 3, because the image has 3 colour channels - red, green and blue. For a black and white image, the depth is one (levels of grey). The convolution operation extracts patches from the input feature map and applies the same transformation to all of these patches, producing an <b>output feature map</b>. 

This output feature map is still a 3D tensor: it has a width and height. Its depth can be <u>arbitrary</u>, because the output depth is a parameter of the layer, and the different channels in that depth axis no longer stand for RGB input, but stand for <b>filters</b>. Filters encode specific aspects of the input data.

In the **handwritten-digits** example, the first convolution layer takes a feature map of `(28,28,1)` and outputs a feature map of `(26,26,32)`. (See `model.summary()`). It computes 32 filters over its input. Each of these 32 output channels is of size `26 x 26` of values, which is a <b>response map</b> of the filter over the input, indicating the response of that filter pattern at different locations in the input. This is the meaning of the **feature map** - every dimension in the depth axis is a feature / filter, and the 2D tensor `output[:,:,n]` is the 2D spatial <b>map</b> of the response of this filter over the input.

Convlutions are defined by 2 key parameters:
- Size of the patches extracted from the inputs - the size of the window e.g. 3x3 or 5x5 windows.
- Depth of the output feature map - the number of filters computed by the convolution. This example started with 32 and ended with 64.

In keras, they are passed as the first argument for the layer:

```python
# 1st layer
model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
# 3rd and final layer
model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))```

A convolution works by <u>sliding</u> these windows of size 3x3 or 5x5 over the 3D input feature map, stopping at every possible location and extracting the 3D patch of surrounding features of shape `(window_height, window_width, input_depth)`. Each such 3D patch is then transformed (via a tensor operation with the same learned weight matrix, called the <b>convolution kernel</b>) into a 1D vector of shape `(output_depth,)`. Every spatial location in the output feature map corresponds to the same location in the input feature map. So if the vector output is `[i,j,:]` then the vector input is `[i-1:i+1,j-1:j+1,:]` for a 3x3 window, `[i-2:i+2,j-2:j+2,:]` for a 5x5 window and so on.

Using images, this is what it looks like:

<img src="img5g.png" width="500"/>

<b><u>BORDER EFFECTS AND PADDING</u></b>

These ideas can be seen intuitively when using sliding windows. For a 5x5 image, we can form 9 3x3 windows.
<img src="img5f.png" width="350"/>

If we want to retain the 5x5 output feature map, simply use <b>padding</b> to add surrounding rows and columns.
<img src="img5e.png" width="450"/>

In `Conv2D` layers, padding is configurable via the `padding` argument, which can take either `valid` (default) which means no padding or `same` which means to pad such that the output has the same width and height as the input.

<b><u>CONVOLUTION SLIDES</u></b>

The idea of <b>strides</b> can also be seen using the sliding windows. If the stride is 1, we capture every input window for every step. If the stride increases to 2 then we capture every other window as we slide. 

We can also see that by increasing the stride to 2x2, then we downsample the feature map by a factor of 2 since we only consider every alternate window on both axes. Strided convolutions are rarely used in practice but it's good to be familiar with the concept. To downsample feature maps, we usually use the <b>max-pooling</b> operation.

<img src="img5d.png" width="450"/>

### The Max-Pooling Operation

In the convnet example, notice that after each `MaxPooling2D` layer, the size of the feature map is halved. That is the function of max-pooling - to aggressively downsample feature maps, much like strided convolutions. 

Max pooling consists of extracting windows from the input feature maps and outputting the max value of each channel. It is conceptually similar to convolution, except that instead of transforming local patches via a learned linear transformation, they are transformed via a hardcoded `max` tensor operation.

Why maxpooling? Consider a network without the maxpooling layer:

In [11]:
model2 = keras.models.Sequential()
model2.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model2.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
model2.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
model2.add(keras.layers.Flatten())
model2.add(keras.layers.Dense(64, activation='relu'))
model2.add(keras.layers.Dense(10, activation='softmax'))
print(model2.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 22, 22, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 30976)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                1982528   
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 2,038,922
Trainable params: 2,038,922
Non-trainable params: 0
____________________________________________

Notice the following problems:

1. The total no. of parameters is 2.03M, compared to about 93K (about 4.5%) This is <b>far too large</b> for a small model and will result in overfitting.
2. <b>It is not conducive to learn a spatial hierarchy of features.</b> Recall that spatial hierarchy is learning general patterns of the whole image using local patterns. Without maxpooling, the 3x3 windows in the 3rd layer will only contain information coming from the 7x7 windows from the first layer (local learning). Learning is very localised to that small window. The generalised patterns learned by this convnet will still be very small with regard to the initial input, which may not be enough to classify the digits. We need the features from the last convolution layer to contain information about the totality of the input.

Downsampling is used to reduce the number of feature-map coefficients to process, as well as to induce spatial filter hierarchies by making successive convolution layers look at increasingly large windows.

Beyond maxpooling, we have introduced strides earlier to perform downsampling. Another way can be average pooling, where each local input patch is transformed by taking the average value of each channel over the patch, rather than the max.

### RECAP

There are many terms in this chapter. Some of the key terms that should be easily able to be defined include:
- convnets
- channels/depth axis
- translation-invariant
- spatial hierarchy
- Convolution layer, Maxpooling layer
- feature map
- response map
- downsampling
- strides

Additional References / Resources:

(1) - https://github.com/lutzroeder/Netron

(2) - https://stats.stackexchange.com/questions/208936/what-is-translation-invariance-in-computer-vision-and-convolutional-neural-netwo